# Overview

We need to refine the way users specify colors, to simplify the public API and eliminate inconsistencies.  The challenge is to handle all of the ways in which a user might want to specify per-series / per-datum colors, while avoiding potential ambiguities.

**Use-cases**

* Specify one explicit color for everything.
        toyplot.plot(x, y, color="red")
* Specify an explicit color per series.
        toyplot.plot(x, y, color=["red", "cinnamon", "cardinal"])
* Specify a palette for the series.
        toyplot.plot(x, y, color=toyplot.color.brewer("reds"))
* Specify a colormap for the series.
        toyplot.plot(x, y, color=toyplot.color.LinearMap(toyplot.color.brewer("reds")))
* Specify per-datum scalar values and have them mapped.
        toyplot.plot(x, y, color=temperature)
* Specify per-datum scalar values and have them mapped, but override a few "special" values.
        colors = toyplot.color.broadcast(temperature)
        colors[numpy.argsort(temperature)[0]] = "blue"
        colors[numpy.argsort(temperature)[-1]] = "red"
        toyplot.plot(x, y, color=colors)
* Specify explicit per-datum color values for everything (i.e. map them yourself).
        toyplot.plot(x, y, color=mycolors)

**Mark Parameters**

* &#x2713; bars -> color, opacity
    * TODO: title
* &#x2713; fill -> color, opacity
    * TODO: title
* graph -> vcolor, ecolor
    * TODO: vfill, vstroke, vlfill, vlstroke, elfill, elstroke
* &#x2713; hlines -> color, opacity
    * TODO: title
* &#x2713; plot -> color, opacity, mfill
    * TODO: title, width, mopacity, mstroke, mtitle, mlfill, mlstroke
* &#x2713; rects -> color, opacity
    * TODO: title, stroke
* &#x2713; scatterplot -> color
    * TODO: opacity, title, mstroke, mlfill, mlstroke
* &#x2713; text -> color, opacity
    * TODO: title, stroke
* &#x2713; vlines -> color, opacity
    * TODO: title

In [1]:
import numpy
import toyplot